In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.utils import to_categorical 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/digit-recognizer/train.csv')
test_df = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
train_label = train_df['label']

In [ ]:
train_label = to_categorical(train_label , num_classes=10)

In [ ]:
train_df.shape

In [ ]:
train_data = (train_df.iloc[:,1:].values.astype('float32') /255.0).reshape(-1,28,28,1)
test_data = (test_df.iloc[:,:].values.astype('float32') /255.0).reshape(-1,28,28,1)

In [ ]:
train_df.iloc[0:2,1:]

In [ ]:
import matplotlib.pyplot as plt 

plt.figure(figsize = (10,10))

def show_im(data , label):
    plt.imshow(data)
    plt.title(label)
    plt.axis('off')
    plt.show()
    
    
show_im((train_df.iloc[1:2,1:].values.astype('float32')).reshape(28,28,1) ,train_df['label'][1])

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_val , y_train ,y_val = train_test_split(train_data,train_label  ,test_size =.3 , random_state =42)

In [ ]:
from tensorflow.keras import Sequential , layers 

In [ ]:
model  = Sequential()
model.add (layers.Conv2D(32,3,input_shape=(28,28,1) ,activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add (layers.Conv2D(16,3,activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128 ,activation = 'relu'))
model.add(layers.Dense(10,activation='softmax'))


In [ ]:
model.compile(
optimizer="Adam" ,
loss="categorical_crossentropy",
metrics =['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='loss' ,patience=3)

history = model.fit(
x_train, y_train ,
    validation_data=(x_val , y_val),
    epochs =20,
    callbacks = [callback])

In [ ]:
model.evaluate(x_val ,y_val)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
predictions  = model.predict(test_data)

In [ ]:
predictions = np.argmax(predictions , axis =1)

In [ ]:
submission=pd.read_csv('../input/digit-recognizer/sample_submission.csv')
submission['Label']=predictions
submission.to_csv('prediction4.csv', index=False)